# QSCOUT Optimizations with Cirq

<a target="_blank" href="https://colab.research.google.com/github/SupertechLabs/client-superstaq/blob/main/docs/source/optimizations/qscout/qscout_css.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>
<a target="_blank" href="https://mybinder.org/v2/gh/SupertechLabs/client-superstaq/HEAD?labpath=docs/source/optimizations/qscout/qscout_css.ipynb">
  <img src="https://mybinder.org/badge_logo.svg" alt="Launch Binder" align="left" style="margin-left: 10px;"/>
</a>

Below is a brief tutorial on SuperstaQ optimizations for Quantum Scientific Computing Open User Testbed (QSCOUT), a trapped ion quantum computing testbed at Sandia National Laboratory. For more information on QSCOUT, visit their website [here](https://docs-superstaq.readthedocs.io/en/stable/get_started/basics/basics_css.html#Set-up-access-to-Superstaq's-API).

## Imports and API Token

This example tutorial notebook uses `cirq-superstaq`, our Superstaq client for Cirq; you can try it out by running `pip install cirq-superstaq`:

In [1]:
%pip install cirq-superstaq

In [2]:
# Required imports
import numpy as np
import cirq
import cirq_superstaq as css

# Optional imports
import os  # used if setting a token as an environment variable

To interface Superstaq via Cirq, we must first instantiate a service provider in `cirq-superstaq` with `Service()`. We then supply a Superstaq API token (or key) by either providing the API token as an argument of `css.Service()` or by setting it as an environment variable (see more details [here](https://docs-superstaq.readthedocs.io/en/latest/get_started/basics.html#Set-up-access-to-SuperstaQ%E2%80%99s-API))

In [3]:
# Get cirq superstaq service for Superstaq backend
service = css.Service()

## Single Circuit Compilation

Let us start by creating an example qiskit circuit that we will then compile and optimize for the QSCOUT trapped-ion testbed at Sandia National Laboratories.

In [4]:
# Create a two-qubit cirq circuit
theta = np.random.uniform(0, 4 * np.pi)
qubits = cirq.LineQubit.range(2)
circuit1 = cirq.Circuit(
    cirq.CX(qubits[0], qubits[1]),
    cirq.rz(theta).on(qubits[1]),
    cirq.CX(qubits[0], qubits[1]),
    cirq.measure(qubits[0], qubits[1]),
)

# Draw circuit for visualization
print(circuit1)

0: ───@─────────────────@───M───
      │                 │   │
1: ───X───Rz(-0.579π)───X───M───


Using the same circuit from above as input, we will now compile it for QSCOUT and visualize the differences by printing the compiled circuit. 

In [5]:
# Compile with qscout compile
compiler_output = service.qscout_compile(circuit1)

# Call circuit from the compiler output to get the corresponding output circuit
output_circuit = compiler_output.circuit

# Visualize the compiled circuit
print(compiler_output.circuit)

0: ───PhX(-0.5)^0.5───Z───MS(0.21π)───PhX(-0.5)^0.5───M───
                          │                           │
1: ───PhX(-0.5)^0.5───Z───MS(0.21π)───PhX(-0.5)^0.5───M───


The resulting output is now a circuit compiled to QSCOUT's native operations. And with Superstaq compilation, you can also get the corresponding Jaqal (see [Just another quantum assembly language](https://arxiv.org/abs/2008.08042)) program for the compiled circuit by running the following:

In [6]:
# Get jaqal program
print(compiler_output.jaqal_program)

from qscout.v1.std usepulses *

register allqubits[2]

prepare_all
<
	R allqubits[0] -1.5707963267948966 1.5707963267948966
	R allqubits[1] -1.5707963267948966 1.5707963267948966
>
<
	Rz allqubits[0] -3.141592653589793
	Rz allqubits[1] -3.141592653589793
>
MS allqubits[0] allqubits[1] 0 1.3229450958830156
<
	R allqubits[0] -1.5707963267948966 1.5707963267948966
	R allqubits[1] -1.5707963267948966 1.5707963267948966
>
measure_all



## Multiple Circuits Compilation

All the functionalities we have seen so far can also be used on a multiple circuits input as well. To illustrate this, let us create a different, example two-qubit circuit (say, a Bell-state circuit):

In [7]:
# Create second circuit
qubits = cirq.LineQubit.range(2)
circuit2 = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CX(qubits[0], qubits[1]),
    cirq.measure(qubits[0], qubits[1]),
)

# Printing second circuit for visualization
print(circuit2)

0: ───H───@───M───
          │   │
1: ───────X───M───


By passing multiple circuits as a list to the `qscout_compile` endpoint, we can compile all of them individually with a single call to `qscout_compile`. This will return all the corresponding compiled circuits and Jaqal programs back as a list, like so:  

In [8]:
# Create list of circuits
circuit_list = [circuit1, circuit2]

# Compile a list of circuits and their respective jaqal programs
compiler_output_list = service.qscout_compile(circuit_list)
jaqal_output_list = compiler_output_list.jaqal_programs

In [9]:
# To get the list of compiled output circuits from the compiler output list, call `circuits` instead of just `circuit` that is called for a single circuit input
output_circuits = compiler_output_list.circuits

# Visualize and get the jaqal program of the first compiled circuit
print("-" * 65, "\n")
print(jaqal_output_list[0])
print("-" * 65, "\n")
print("Compiled circuit 1 \n")
print(output_circuits[0])

----------------------------------------------------------------- 

from qscout.v1.std usepulses *

register allqubits[2]

prepare_all
<
	R allqubits[0] -1.5707963267948966 1.5707963267948966
	R allqubits[1] -1.5707963267948966 1.5707963267948966
>
<
	Rz allqubits[0] -3.141592653589793
	Rz allqubits[1] -3.141592653589793
>
MS allqubits[0] allqubits[1] 0 1.3229450958830156
<
	R allqubits[0] -1.5707963267948966 1.5707963267948966
	R allqubits[1] -1.5707963267948966 1.5707963267948966
>
measure_all

----------------------------------------------------------------- 

Compiled circuit 1 

0: ───PhX(-0.5)^0.5───Z───MS(0.21π)───PhX(-0.5)^0.5───M───
                          │                           │
1: ───PhX(-0.5)^0.5───Z───MS(0.21π)───PhX(-0.5)^0.5───M───


In [10]:
# Visualize and get the jaqal program of second compiled circuit
print("-" * 65, "\n")
print(jaqal_output_list[1])
print("-" * 65, "\n")
print("Compiled circuit 2 \n")
print(output_circuits[1])

----------------------------------------------------------------- 

from qscout.v1.std usepulses *

register allqubits[2]

prepare_all
<
	R allqubits[0] 3.141592653589793 3.141592653589793
	R allqubits[1] 3.141592653589793 3.141592653589793
>
MS allqubits[0] allqubits[1] 0 -1.5707963267948966
<
	R allqubits[0] -1.5707963267948966 1.5707963267948966
	R allqubits[1] 3.141592653589793 1.5707963267948966
>
measure_all

----------------------------------------------------------------- 

Compiled circuit 2 

0: ───PhX(1)───MS(-0.25π)───PhX(-0.5)^0.5───M───
               │                            │
1: ───PhX(1)───MS(-0.25π)───PhX(1)^0.5──────M───


## Entangling Basis Compilation

When compiling to the QSCOUT gateset, we can also specify the type of entangling basis gate to utilize during the compilation -- either a $XX$ or $ZZ$. Let's consider the first circuit example for earlier but with a different, random $R_Z$ angle and single measurement:

In [11]:
# Create circuit
theta = np.random.uniform(0, 4 * np.pi)
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.CX(qubits[0], qubits[1]),
    cirq.rz(theta).on(qubits[1]),
    cirq.CX(qubits[0], qubits[1]),
    cirq.measure(qubits[0], qubits[1]),
)

# Print circuit for visualization
print(circuit)

0: ───@───────────────@───M───
      │               │   │
1: ───X───Rz(1.55π)───X───M───


In [12]:
# Compile with XX entangling basis
compiler_output = service.qscout_compile(circuit, base_entangling_gate="xx")

# Visualize and get jaqal program of the compiled circuit
print("-" * 70, "\n")
print(compiler_output.jaqal_program)
print("-" * 70, "\n")
print(compiler_output.circuit)

---------------------------------------------------------------------- 

from qscout.v1.std usepulses *

register allqubits[2]

prepare_all
<
	R allqubits[0] -1.5707963267948966 1.5707963267948966
	R allqubits[1] -1.5707963267948966 1.5707963267948966
>
<
	Rz allqubits[0] -3.141592653589793
	Rz allqubits[1] -3.141592653589793
>
MS allqubits[0] allqubits[1] 0 -1.4140915932358178
<
	R allqubits[0] -1.5707963267948966 1.5707963267948966
	R allqubits[1] -1.5707963267948966 1.5707963267948966
>
measure_all

---------------------------------------------------------------------- 

0: ───PhX(-0.5)^0.5───Z───MS(-0.225π)───PhX(-0.5)^0.5───M───
                          │                             │
1: ───PhX(-0.5)^0.5───Z───MS(-0.225π)───PhX(-0.5)^0.5───M───


As we can see above, we get the same compiled gate structure as before. This is because, by default, the $XX$ interaction is used, and we observe that it uses an $MS$ gate as the base entangling gate in the compiled circuit. Similarly, let's take a look at the compiled circuit if we now specify the compiler to use the $ZZ$ interaction instead:

In [13]:
# Compile with zz entangling basis
compiler_output = service.qscout_compile(circuit, base_entangling_gate="zz")

# Get jaqal program and print circuit
print("-" * 70, "\n")
print(compiler_output.jaqal_program)
print("-" * 70, "\n")
print(compiler_output.circuit)

---------------------------------------------------------------------- 

from qscout.v1.std usepulses *

register allqubits[2]

prepare_all
ZZ allqubits[0] allqubits[1] -1.4140915932358178
measure_all

---------------------------------------------------------------------- 

0: ───ZZ─────────M───
      │          │
1: ───ZZ^-0.45───M───


Looking at the compiled circuit above, we can see that by specifying the $ZZ$ entangling basis, we use $R_{ZZ}$ gate as the base entangling gate and the remaining single-qubit gates are compiled and optimized accordingly. 

## Swap Mirroring

In addition to specifying the type of base entangling gate to use for the compilation and optimization, we can also specify whether to use mirror swapping to help reduce the two-qubit gate overhead of the circuit. By default, it is not enabled; but we will see the differences in circuit compilation by compiling a random quantum volume model circuit. 

### Create random Quantum Volume (QV) circuit

In [14]:
from cirq.contrib import quantum_volume

In [15]:
# Generate a random QV circuit
circ = quantum_volume.generate_model_circuit(
    num_qubits=2, depth=2, random_state=np.random.RandomState(seed=123)
)
circ.append([cirq.measure(*cirq.LineQubit.range(2))])

### Compile without swap mirroring

In [16]:
# Compile with no swap mirroring
output_nsm = service.qscout_compile(circ, mirror_swaps=False, base_entangling_gate="zz")

# Visualize the circuit
circ_nsm = output_nsm.circuit
print(circ_nsm)

0: ───PhX(-0.548)^0.918───ZZ──────────PhX(-0.184)^0.5───ZZ───────────PhX(0.316)^0.5────ZZ─────────PhX(0.385)^0.775──────M───
                          │                             │                              │                                │
1: ───PhX(0.408)^0.551────ZZ^-0.476───PhX(-0.702)^0.5───ZZ^(-4/13)───PhX(-0.202)^0.5───ZZ^0.052───PhX(-0.259)^(11/16)───M───


### Compile with swap mirroring

In [17]:
# Compile with swap mirroring
output_wsm = service.qscout_compile(circ, mirror_swaps=True, base_entangling_gate="zz")

# Visualize the circuit
circ_wsm = output_wsm.circuit
print(circ_wsm)

0: ───PhX(-0.409)^0.426────ZZ──────────PhX(0.0567)^0.5───ZZ──────────PhX(0.557)^0.5───ZZ──────────PhX(-0.254)^0.547───M────────────────
                           │                             │                            │                               │
1: ───PhX(-0.0731)^0.391───ZZ^-0.448───PhX(-0.128)^0.5───ZZ^-0.193───PhX(0.372)^0.5───ZZ^-0.024───PhX(0.094)^0.544────M('q(0),q(1)')───


With the use of swap mirroring, we note that the keys associated with each measurement have now been swapped in the compiled circuit. This is also displayed with the measurement indices in the above compiled circuit compared to the previous compiled circuit that does not have swap mirroring enabled.

### Return final logical to physical qubit mapping

In [18]:
# Return final qubit map for non swap-mirrored circuit
map_nsm = output_nsm.final_logical_to_physical
print("Non swap-mirrored mapping:", map_nsm)

# Return final qubit map for swap-mirrored circuit
map_wsm = output_wsm.final_logical_to_physical
print("Swap-mirrored mapping:", map_wsm)

Non swap-mirrored mapping: {cirq.LineQubit(0): cirq.LineQubit(0), cirq.LineQubit(1): cirq.LineQubit(1)}
Swap-mirrored mapping: {cirq.LineQubit(0): cirq.LineQubit(1), cirq.LineQubit(1): cirq.LineQubit(0)}


## Using Superstaq Simulator

Lastly, we will go over how to submit a circuit to a backend and simulate it. This feature is available to free trial users, and can be done by passing the `"dry-run"` method parameter when calling `create_job()` to instruct Superstaq to simulate the circuit. 

In [19]:
# Example Bell state circuit
qubits = cirq.LineQubit.range(2)
qc = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CX(qubits[0], qubits[1]),
    cirq.measure(qubits[0], qubits[1]),
)

# Get qscout backend from service provider
job = service.create_job(
    circuit=qc, repetitions=100, target="sandia_qscout_qpu", method="dry-run"
)  # specify "dry-run" as the method to run Superstaq simulation

# Get the counts from the measurement
print(job.counts())

{'00': 58, '11': 42}
